In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import IPython
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
from tqdm import tqdm
import tensorflow as tf

from nara_wpe.tf_wpe import wpe
from nara_wpe.utils import stft, istft, get_stft_center_frequencies
from nara_wpe import project_root

In [ ]:
stft_options = dict(size=512, shift=128)

# Minimal example with random data

In [ ]:
def aquire_audio_data():
    D, T = 4, 10000
    y = np.random.normal(size=(D, T))
    return y

In [ ]:
y = aquire_audio_data()
Y = stft(y, **stft_options).transpose(2, 0, 1)
with tf.Session() as session:
    Y_tf = tf.placeholder(
        tf.complex128, shape=(None, None, None))
    Z_tf = wpe(Y_tf)
    Z = session.run(Z_tf, {Y_tf: Y})
z_tf = istft(Z.transpose(1, 2, 0), size=stft_options['size'], shift=stft_options['shift'])

# Example with real audio recordings
WPE estimates a filter to predict the current reverberation tail frame from K time frames which lie 3 (delay) time frames in the past. This frame (reverberation tail) is then subtracted from the observed signal.

### Setup

In [ ]:
channels = 8
sampling_rate = 16000
delay = 3
iterations = 5
taps = 10

### Audio data
Shape: (frames, channels)

In [ ]:
file_template = 'AMI_WSJ20-Array1-{}_T10c0201.wav'
signal_list = [
    sf.read(str(project_root / 'data' / file_template.format(d + 1)))[0]
    for d in range(channels)
]
y = np.stack(signal_list, axis=0)
IPython.display.Audio(y[0], rate=sampling_rate)

### STFT
For simplicity reasons we calculate the STFT in Numpy and provide the result in form of the Tensorflow feed dict.

In [ ]:
Y = stft(y, **stft_options).transpose(2, 0, 1)

### iterative WPE
A placeholder for Y is declared. The wpe function is fed with Y via the Tensorflow feed dict. Finally, an inverse STFT in Numpy is performed to obtain a dereverberated result in time domain.

In [ ]:
from nara_wpe.tf_wpe import get_power
with tf.Session()as session:
    Y_tf = tf.placeholder(tf.complex128, shape=(None, None, None))
    Z_tf = wpe(Y_tf, taps=taps, iterations=iterations)
    Z = session.run(Z_tf, {Y_tf: Y})
z = istft(Z.transpose(1, 2, 0), size=stft_options['size'], shift=stft_options['shift'])
IPython.display.Audio(z[0], rate=sampling_rate)

# Power spectrum
Before and after applying WPE

In [ ]:
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(20, 8))
im1 = ax1.imshow(20 * np.log10(np.abs(Y[:, 0, 200:400])), origin='lower')
ax1.set_xlabel('')
_ = ax1.set_title('reverberated')
im2 = ax2.imshow(20 * np.log10(np.abs(Z[:, 0, 200:400])), origin='lower')
_ = ax2.set_title('dereverberated')
cb = fig.colorbar(im1)